In [82]:
import sys
sys.path.append('..')
import os, pickle, torch
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from sklearn.preprocessing import LabelEncoder
from src.feature import precursor_count, parse_feature

root_f = '/home/jhyang/WORKSPACES/MODELS/isyn/VAE_FCNN'
root_g = '/home/jhyang/WORKSPACES/MODELS/isyn/VAE_graph'
root_t = '/home/jhyang/WORKSPACES/MODELS/isyn/VAE_test'

In [69]:
output = {}
for dir in os.listdir(root_t):
    if not os.path.isfile(os.path.join(root_t, dir, 'output.valid.pkl')): continue
    output[dir] = {
        'test' :pickle.load(open(os.path.join(root_t, dir, 'output.test.pkl'), 'rb')),
        'valid':pickle.load(open(os.path.join(root_t, dir, 'output.valid.pkl'), 'rb')),
    }

In [74]:
out = output['fcnn_active_composit_batch_0128_mdim_32_128_4']

In [75]:
f1s = {
    'fc_f1_mi' : f1_score(out['valid']['pred'].argmax(1), out['valid']['label'], average='micro') ,
    'fc_f1_ma' : f1_score(out['valid']['pred'].argmax(1), out['valid']['label'], average='macro') ,
#    'gc_f1_mi' : f1_score(conv_c['valid']['pred'].argmax(1), conv_c['valid']['label'], average='micro') ,
#    'gc_f1_ma' : f1_score(conv_c['valid']['pred'].argmax(1), conv_c['valid']['label'], average='macro') ,
}
for k,v in f1s.items():
    print('{:10s} : {:.5f}'.format(k,v))

fc_f1_mi   : 0.99968
fc_f1_ma   : 0.98636


In [83]:

path = f'{root_t}/cls/fcnn_active_composit_batch_0128_mdim_32_128_4/'


In [12]:
#np.unique(output['convolution_active_composit_batch_0128_mdim_32_128_4_cls_case_09']['test']['label'], return_counts=True)
#valid = output['convolution_active_composit_batch_0128_mdim_32_128_4_cls_case_09']['valid']
#test  = output['convolution_active_composit_batch_0128_mdim_32_128_4_cls_case_09']['test']
for i, (comp, cnt) in enumerate(precursor_count.items()):
    s1 = '{:4d} {:10s} {:5d} | FCNN 0 | '.format(i, '-'.join(comp), cnt)
    s2 = '{:21s} | FCNN 1 | '.format(' ')
    for s, out in zip([s1, s2], [fcnn_0, fcnn_1]):
        f1s = []
        for d in ['valid','test']:
            m1 = out[d]['label'] == i
            m2 = out[d]['pred'].argmax(1) == i
            tp = (m1 & m2).sum()
            tn = (~m1 & ~m2).sum()
            fp = (~m1 & m2).sum()
            fn = (m1 & ~m2).sum()
            if m1.sum() != 0:
                precision = tp / (tp + fp)*100 if tp + fp !=0 else 0
                recall = tp / (tp + fn)*100 if tp + fn != 0 else 0
                f1_score = 2 * tp / (2 * tp + fp + fn) * 100 if tp + fp + fn != 0 else 0
                f1s.append(f1_score)
                s += ' {:5d} {:5d} {:8.3f} {:8.3f} {:8.3f} |'.format(m1.sum(), m2.sum(), precision, recall, f1_score)
            else:
                s += ' '*40 + '|'
        if len(s.replace('|','').strip()) < 30:
            continue
    #    if np.min(f1s) < 80: print(s)
        print(s)
#    print()

   0 O-Ti        8989 | FCNN 0 |    716   717   99.721   99.860   99.791 |   535   542   98.708  100.000   99.350 |
                      | FCNN 1 |    716   717   99.721   99.860   99.791 |   535   541   98.706   99.813   99.257 |
   1 C-O-Sr      7094 | FCNN 0 |    540   590   91.525  100.000   95.575 |   365   396   91.919   99.726   95.664 |
                      | FCNN 1 |    540   589   91.681  100.000   95.660 |   365   394   92.132   99.452   95.652 |
   2 C-O-Ba      7071 | FCNN 0 |    554   591   93.739  100.000   96.769 |   335   359   93.036   99.701   96.254 |
                      | FCNN 1 |    554   588   94.218  100.000   97.023 |   335   358   93.575  100.000   96.681 |
   3 O-La        6246 | FCNN 0 |    526   566   92.933  100.000   96.337 |   292   326   89.571  100.000   94.498 |
                      | FCNN 1 |    526   555   94.595   99.810   97.132 |   292   310   93.548   99.315   96.346 |
   4 O-Fe        6035 | FCNN 0 |    537   568   94.366   99.814   97.014

In [26]:
from src.feature import feature_to_composit, feature_to_ligand_index, ligand_composit_feature, ligand_label, ligand_index, ligand_vector
from src.utils import find_nearest, squared_error, cosin_similarity
from sklearn.metrics import confusion_matrix

prd = output['conv_active_composit_batch_0256_mdim_16_032_2_case_09']['valid']['pred']
inp = output['conv_active_composit_batch_0256_mdim_16_032_2_case_09']['valid']['input']

inp_lbls = []
prd_lbls = []
for i, p in zip(inp, prd):
    i_i, sser, csim = find_nearest(i.reshape(1,-11), ligand_vector)
    i_p, sser, csim = find_nearest(p.reshape(1,-11), ligand_vector)
    inp_lbls.append(i_i[0])
    prd_lbls.append(i_p[0])
cm = confusion_matrix(inp_lbls, prd_lbls)

In [43]:
for i, (idx, cnt) in enumerate(zip(*np.unique(inp_lbls+prd_lbls, return_counts=True))):
#    print(ligand_index[idx], cnt, cm[i,i] / cm[i].sum(), cm[i,i] / cm[:,i].sum())
    print('{:15s} {:10d} ({:.5f}) {:10d} ({:.5f})'.format(ligand_index[i], cm[i].sum(), cm[i].sum()/len(inp_lbls), cm[:, i].sum(), cm[:, i].sum()/len(inp_lbls)))
#for i in range(79):
#    print('{:15s} {:10.0f} {:8.4f} {:8.4f}'.format(ligand_index[i], cm[i, i], cm[i, i] / cm[i].sum(), cm[i, i] / cm[:, i].sum()))

Metal-O               8486 (0.42915)       9203 (0.46541)
Metal-C-O             2493 (0.12607)       2393 (0.12102)
Metal                 7103 (0.35921)       7803 (0.39461)
Metal-H-O              144 (0.00728)          0 (0.00000)
Metal-N-O               95 (0.00480)          0 (0.00000)
Se                     294 (0.01487)          0 (0.00000)
H-N-O-P                189 (0.00956)        126 (0.00637)
S                      203 (0.01027)          0 (0.00000)
Metal-H-C-O             81 (0.00410)          0 (0.00000)
Metal-F                147 (0.00743)          0 (0.00000)
Metal-S                140 (0.00708)         28 (0.00142)
Metal-N                 36 (0.00182)          0 (0.00000)
Metal-Cl                37 (0.00187)          0 (0.00000)
Metal-H-N-O             53 (0.00268)          0 (0.00000)
Metal-O-P               39 (0.00197)          0 (0.00000)
Metal-H-O-P             32 (0.00162)          0 (0.00000)
P                       33 (0.00167)        110 (0.00556)
Metal-Se      

In [236]:
tol = 0.5
input_index = np.array(feature_to_ligand_index(o['input']))
pred_index = np.array(feature_to_ligand_index(o['pred'], tol=0.5))
input_label = np.array([ligand_index[i] for i in input_index])
pred_label = np.array([ligand_index[i] if i != -1 else 'Fail' for i in pred_index])

In [244]:
import gzip, pickle
with gzip.open('../data/unique_ligand.pkl.gz','rb') as f:
    unique_ligand = pickle.load(f)
unique_ligand

{('O',): {'count': 86539,
  'metals': {('Y',): {'count': 3101,
    'case': [{'composition': {'Y': 0.4, 'O': 0.6},
      'years': [1979,
       1979,
       1986,
       1995,
       1995,
       1996,
       1996,
       1996,
       1996,
       1997,
       1997,
       1997,
       1997,
       1998,
       1998,
       1998,
       1998,
       1998,
       1998,
       1998,
       1998,
       1998,
       1998,
       1998,
       1998,
       1998,
       1998,
       1998,
       1998,
       1998,
       1998,
       1998,
       1998,
       1998,
       1998,
       1998,
       1998,
       1998,
       1998,
       1998,
       1998,
       1998,
       1998,
       1998,
       1998,
       1998,
       1998,
       1999,
       1999,
       1999,
       1999,
       1999,
       1999,
       1999,
       1999,
       1999,
       1999,
       1999,
       1999,
       1999,
       2000,
       2000,
       2000,
       2000,
       2000,
       2000,
       2000,
      

In [102]:
ligands = {}
precursors = {}
for ligand, _ligand_data in unique_ligand.items():
    for metal, _metal_data in _ligand_data['metals'].items():
        precursor = metal, ligand
        c = _metal_data['count']
        if len(metal) == 0:
            if precursor not in ligands.keys():
                ligands[precursor] = 0
                precursors[precursor] = 0
            ligands[precursor] += c
            precursors[precursor] += c
        else:
            if precursor not in precursors.keys():
                precursors[precursor] = 0
            if (('Metal',), ligand) not in ligands.keys():
                ligands[(('Metal',), ligand)] = 0
            ligands[(('Metal',), ligand)] += c
            precursors[precursor] += c
ligands = {k:v for k,v in sorted(ligands.items(), key=lambda x: x[1], reverse=True)}
precursors = {k:v for k,v in sorted(precursors.items(), key=lambda x: x[1], reverse=True)}
ligands.update({((),()):0})

precursors.update({((),()):0})


In [107]:
from src.feature import active_composit_feature, ligand_composit_feature

ligand_vector = np.vstack([ligand_composit_feature({e if e != 'Metal' else 'Li':1 for e in k[0] + k[1]}, dtype=float) for k in ligands.keys()])
precursor_vector = np.vstack([active_composit_feature({e:1 for e in k[0] + k[1]}, dtype=float) for k in precursors.keys()])

def parse_feature(feat_vec, csim_cut = 0.5, sser_cut = 1.0, to_string=False):
    out = []
    if feat_vec.shape[-1] == 12:
        ref = ligand_vector
        chrs = np.array(['-'.join(k[0] + k[1]) for k in ligands.keys()] + ['Unknown'])
    elif feat_vec.shape[-1] == 97:
        ref = precursor_vector
        chrs = np.array(['-'.join(k[0] + k[1]) for k in precursors.keys()] + ['Unknown'])
    else:
        raise ValueError('Not supported feature type')

    for idx, sser, csim in zip(*find_nearest(feat_vec, ref)):
        if (sser > sser_cut) or (csim < csim_cut):
            out.append(-1)
        else:
            out.append(int(idx))
    if to_string:
        return chrs[out]
    else:
        return out
    

_inp_lbls = parse_feature(inp)
_prd_lbls = parse_feature(prd)

In [36]:
pred_index[4554], input_index[4554]

(0, 0)

In [49]:
for i_inp in sorted(np.unique(input_index), key=lambda x: ligand_index[x]):
    m = input_index == i_inp
    a = np.sum(pred_index[m] == i_inp)
    t = np.sum(m)
    print('{:15s} {:5d} {:5d} {:8.3f}'.format(ligand_index[i_inp], t, a, 100*a/t))
    
#    print(sorted(zip(*np.unique(np.array(pred_label)[m], return_counts=True)), key=lambda x: x[1], reverse=True))

C                   7     7  100.000
H-C-N-O             2     0    0.000
H-C-O               6     0    0.000
H-C-S-Br            3     0    0.000
H-C-S-Cl            3     0    0.000
H-C-S-I             4     0    0.000
H-N-Cl             19    12   63.158
H-N-F               2     0    0.000
H-N-O-P            18    18  100.000
H-N-O-S             5     0    0.000
H-O-P               8     0    0.000
Metal            1518  1466   96.574
Metal-Br           10     2   20.000
Metal-C             4     0    0.000
Metal-C-O        1572  1448   92.112
Metal-C-S           1     0    0.000
Metal-Cl           23     5   21.739
Metal-F            36    14   38.889
Metal-H-C-N-Br      1     0    0.000
Metal-H-C-N-Cl      1     0    0.000
Metal-H-C-N-I       1     0    0.000
Metal-H-C-O        90     0    0.000
Metal-H-F           1     0    0.000
Metal-H-N-O        28     3   10.714
Metal-H-O          97    15   15.464
Metal-H-O-P        10     0    0.000
Metal-H-O-S         2     0    0.000
M

In [66]:
import run_fcvae_class as R

#R.parser.parse_args('--logging')
#R.args
#R.logging = True
#R.epochs = 5
#R.main(R.args)

usage: ipykernel_launcher.py [-h] [--hidden_dim HIDDEN_DIM]
                             [--hidden_layers HIDDEN_LAYERS]
                             [--latent_dim LATENT_DIM]
                             [--batch_size BATCH_SIZE] [--epochs EPOCHS]
                             [--early_stop EARLY_STOP]
                             [--logging_interval LOGGING_INTERVAL]
                             [--target_feature_type TARGET_FEATURE_TYPE]
                             [--logging]
ipykernel_launcher.py: error: unrecognized arguments: --f=/home/jhyang/.local/share/jupyter/runtime/kernel-v2-245045jJs10vwAO5mJ.json


SystemExit: 2